In [1]:
!pip install sklearn_crfsuite
!pip install -U 'scikit-learn<0.24'
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 965 kB 2.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.8 MB 2.0 MB/s 
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.5 requires scikit-learn>=1.0.0, but you have scikit-learn 0.23.2 which is incompatible.
imbalanced-learn 0.8.1 requires scikit-learn>=0.24, but you have scikit-learn 0.23.2 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import sklearn_crfsuite
import pandas as pd
import numpy as np
import scipy.stats
import sklearn
import joblib
import nltk

from sklearn_crfsuite.metrics import flat_classification_report, flat_f1_score
from sklearn.metrics import classification_report, make_scorer
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
from sklearn_crfsuite import scorers

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
df_aditamento = pd.read_csv('https://raw.githubusercontent.com/brunoedcf/data_crf_training/main/aditamento_contratual.csv')

In [5]:
class CRF_CrossValidation_Aditamento():

  def __init__(self):

    self.x = None
    self.y = None
    self.labels = [
        'B-PROCESSO', 
        'B-NUM_ADITIVO', 
        'I-NUM_ADITIVO', 
        'B-NUM_AJUSTE', 
        'B-CONTRATANTE', 
        'I-CONTRATANTE', 
        'B-OBJ_ADITIVO', 
        'I-OBJ_ADITIVO', 
        'B-DATA_ESCRITO', 
        'I-DATA_ESCRITO', 
        'B-NOME_RESPONSAVEL', 
        'I-NOME_RESPONSAVEL', 
        'B-CODIGO_SIGGO', 
        'I-NUM_AJUSTE', 
        'I-PROCESSO'
      ]


    self.metrics = None
    self.crf = None

  def get_features(self, sentence):
        
        sent_features = []
        for i in range(len(sentence)):
            # print(sentence[i])
            word_feat = {
                # Palavra atual
                'word': sentence[i].lower(),
                'capital_letter': sentence[i][0].isupper(),
                'all_capital': sentence[i].isupper(),
                'isdigit': sentence[i].isdigit(),
                # Uma palavra antes
                'word_before': '' if i == 0 else sentence[i-1].lower(),
                'word_before_isdigit': '' if i == 0 else sentence[i-1].isdigit(),
                'word_before_isupper': '' if i == 0 else sentence[i-1].isupper(),
                'word_before_istitle': '' if i == 0 else sentence[i-1].istitle(),
                # Uma palavra depois
                'word_after': '' if i+1 >= len(sentence) else sentence[i+1].lower(),
                'word_after_isdigit': '' if i+1 >= len(sentence) else sentence[i+1].isdigit(),
                'word_after_isupper': '' if i+1 >= len(sentence) else sentence[i+1].isupper(),
                'word_after_istitle': '' if i+1 >= len(sentence) else sentence[i+1].istitle(),

                'BOS': i == 0,
                'EOS': i == len(sentence)-1
            }
            sent_features.append(word_feat)
        return sent_features

  def load(self, data_frame):

    self.x = []
    self.y = []

    for i, row in enumerate(data_frame['treated_text']):
      self.x.append(word_tokenize(data_frame['treated_text'][i]))
      self.y.append(data_frame['IOB'][i].split())

    for i in range(len(self.x)):
        self.x[i] = self.get_features(self.x[i])


  def optimize(self):

    crf_optimizer = sklearn_crfsuite.CRF(
      algorithm='lbfgs',
      max_iterations=100,
      all_possible_transitions=True
    )

    params_space = {
      'c1': scipy.stats.expon(scale=0.1),
      'c2': scipy.stats.expon(scale=0.1),
    }
    f1_scorer = make_scorer(
        flat_f1_score,
        average='weighted', 
        labels=self.labels
    )

    rs = RandomizedSearchCV(
        crf_optimizer,
        params_space,
        cv=10,
        verbose=1,
        n_jobs=-1,
        n_iter=30,
        scoring=f1_scorer
    )

    rs.fit(self.x, self.y)
    
    print('best params:', rs.best_params_)
    print('best CV score:', rs.best_score_)

    res = pd.DataFrame(rs.cv_results_)
    res.to_csv('results.csv')

In [6]:
model = CRF_CrossValidation_Aditamento()
model.load(df_aditamento)

In [7]:
model.optimize()

Fitting 10 folds for each of 30 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed: 68.0min
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed: 284.5min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 432.3min finished


best params: {'c1': 0.20305452976423158, 'c2': 0.03200481159541715}
best CV score: 0.88516906365235
